## pytorch_seresnext101-32x4d+kfold+focal loss v3.0


* fold = 4
* epoch = 3
* γ=0.4

In [1]:
# import gc
# gc.collect()

In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
import time
import numpy as np
import math
import pandas as pd
from PIL import Image, ImageOps, ImageFilter
from datetime import datetime
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms
import random
import datetime
import os

import cv2  
from sklearn import preprocessing 
from sklearn.model_selection import KFold
import h5py

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose,RandomContrast,RandomBrightness,Resize
)
import albumentations

In [4]:
from torch.utils.data import DataLoader
from prefetch_generator import BackgroundGenerator

class DataLoaderX(DataLoader):

    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

In [5]:
# os.chdir("../input/pretrained_PyTorch/")
path = os.getcwd()
print(path)

/root/notebooks/imet/iMet-Collection-2019-FGVC6


In [6]:
train = pd.read_csv(path+"/train.csv")
lable = pd.read_csv(path+"/labels.csv")
test = pd.read_csv(path+"/sample_submission.csv")

In [7]:
lable_length = len(lable)
train_length = len(train)
test_length = len(test)
print(train_length)
print(lable_length)
print(test_length)

109237
1103
38801


In [8]:
print(np.array(lable)[397])
print(np.array(lable)[398])
c_length = len(np.array(lable)[:398])
t_length = len(np.array(lable)[398:])
print(c_length)
print(t_length)

[397 'culture::zurich']
[398 'tag::abbies']
398
705


In [9]:
#np.array(test)

In [10]:
# test.head

In [11]:
#np.array(train)

In [12]:
def augment(p=.5):
    return Compose([
        HorizontalFlip(p=0.5),
        OneOf([
            RandomBrightness(0.1, p=1),
            RandomContrast(0.1, p=1),
        ], p=0.3),
        ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
        IAAAdditiveGaussianNoise(p=0.3)
        #Resize(220,220)
    ], p=p)
def strong_aug(p=.5):
    return Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=0.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=p)

aug = augment(0.2)

In [13]:
def creatData(train,lable_length):
    train = np.array(train)
    trainA_data = []
    lab_data = []
    #trainC_data = []
    #trainT_data = []
    for t in range(train_length):
        v = np.zeros(lable_length)
        #print(train[t,1])
        lab = []
        for s in train[t,1].split(" "):
            #print(s)
            v[int(s)] = 1
            lab.append(int(s))
            
#         img = Image.open(path+"/train/"+format(train[t,0])+'.png')  # PIL image
#         img = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
#         img = aug(image = img)['image']
#         #openCV to PIL
#         img = Image.fromarray(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
        
        trainA_data.append([train[t,0],v])
        lab_data.append([train[t,0],np.array(lab)])

        #trainC_data.append([train[t,0],v[:c_length]])
        #trainT_data.append([train[t,0],v[c_length:]])
    return np.array(trainA_data),np.array(lab_data)
    #return np.array(trainA_data)#,np.array(trainC_data),np.array(trainT_data)

In [14]:
train_a,train_lab = creatData(train,lable_length)
#train_a,train_c,train_t = creatData(train,lable_length)
#train_t.shape()

In [15]:
train_lab[0]

array(['1000483014d91860', array([147, 616, 813])], dtype=object)

### Remove the item that only have one feature (not work well)

In [16]:
def dfilter(data,th):
    dfilter = []
    for i in range(len(data)):
        if train_a[i][1].sum()>th:
            dfilter.append(train_a[i])
    return np.array(dfilter)

In [17]:
datafilter = dfilter(train_a,1) # remain feature great then 1

In [18]:
print("amount of image before: {}".format(len(train_a)))

print("amount of image after: {}".format(len(datafilter)))

amount of image before: 109237
amount of image after: 104913


In [19]:
# not use
datafilter = train_a

In [20]:
datafilter[0]

array(['1000483014d91860', array([0., 0., 0., ..., 0., 0., 0.])],
      dtype=object)

In [21]:
class RandomCropIfNeeded(transforms.RandomCrop):
    def __init__(self, height, width, always_apply=False, p=1):
        super(RandomCropIfNeeded, self).__init__(always_apply, p)
        self.height = height
        self.width = width

    def apply(self, img, h_start=0, w_start=0, **params):
        h, w, _ = img.shape
        return F.random_crop(img, min(self.height, h), min(self.width, w), h_start, w_start)

In [22]:
image_resize = 220

#RandomC = RandomCropIfNeeded(image_resize*2,image_resize*2)

data_transforms2 = transforms.Compose([
    RandomCropIfNeeded(image_resize*2,image_resize*2),
    transforms.Resize((image_resize,image_resize)),
    transforms.ToTensor(),
    transforms.Normalize(
            [0.485, 0.456, 0.406], 
            [0.229, 0.224, 0.225])
    ])


data_transforms = transforms.Compose([
    transforms.Resize((image_resize,image_resize)),
    #transforms.RandomResizedCrop(250),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    #transforms.Normalize(
    #        [0.485, 0.456, 0.406], 
    #        [0.229, 0.224, 0.225])
    ])

train_transformer = transforms.Compose([
    transforms.Resize((128,128)),              # resize the image to 
    #transforms.RandomHorizontalFlip(),  # randomly flip image horizontally
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.ToTensor(),           # transform it into a PyTorch Tensor
    #transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
])

In [23]:
# top_transforms = transforms.Compose([
#     augment()['image'],
#     transforms.Resize((image_resize,image_resize)),
#     HorizontalFlip(p=0.5),
#     OneOf([
#         RandomBrightness(0.1, p=1),
#         RandomContrast(0.1, p=1),
#     ], p=0.3),
#     ShiftScaleRotate(shift_limit=0.1, scale_limit=0.0, rotate_limit=15, p=0.3),
#     IAAAdditiveGaussianNoise(p=0.3),
#     #transforms.RandomResizedCrop(250),
#     #transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(
#             [0.485, 0.456, 0.406], 
#             [0.229, 0.224, 0.225])
#     ])

In [24]:
#a,b = load2mem("all_data_109237v1.h5", data_transforms2)

In [25]:
class dataset_h5(torch.utils.data.Dataset):
    def __init__(self, in_file, transform, kdata, aug=None):
        super(dataset_h5, self).__init__()

        self.file = h5py.File(in_file, 'r')
        self.n_images = len(kdata)
        self.kdata = kdata
        self.transform = transform
        print("Load data from .h5 file...")
        t = time.time()
        self.preloadimg = self.file['train_img'][self.kdata,:,:]
        self.preloadleb = self.file['train_labels'][self.kdata,:]
        self.aug = aug
#         self.preloadimg = [a[i] for i in self.kdata]
#         self.preloadleb = b[self.kdata]
        print("Take",datetime.timedelta(seconds = int(time.time()-t)),"seconds")

    def __getitem__(self, index):
        
        input_img = Image.fromarray(self.preloadimg[index])
        
        if self.aug is not None:
            imgv = cv2.cvtColor(np.asarray(input_img),cv2.COLOR_RGB2BGR)
            imgv = self.aug(image = imgv)['image']
            #openCV to PIL
            input_img = Image.fromarray(cv2.cvtColor(imgv,cv2.COLOR_BGR2RGB))
        
        input_img = self.transform(input_img)
        
        input_lab = self.preloadleb[index]
        return input_img,input_lab

    def __len__(self):
        return self.n_images

In [26]:
class trainDataset(Dataset):
    def __init__(self, train_lib, transform,transform2):
        self.filenames = train_lib[:,0]
        self.labels = train_lib[:,1]
        self.transform = transform
        self.transform2 = transform2
        #self.new_feature = 

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = Image.open(path+"/train/"+format(self.filenames[idx])+'.png')  # PIL image
        
        #PIL to openCV
        imgv = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
        imgv = aug(image = imgv)['image']
        #openCV to PIL
        img = Image.fromarray(cv2.cvtColor(imgv,cv2.COLOR_BGR2RGB))
        
        #image= image.filter(ImageFilter.EDGE_ENHANCE)
        #image2 = image.filter(ImageFilter.FIND_EDGES)
        image = self.transform(img)
        image2 = self.transform2(img)
        #return image, self.labels[idx]
        return image,image2, self.labels[idx]


In [27]:
class testDataset(Dataset):
    def __init__(self, test_lib, transform,transform2):
        test_lib = np.array(test_lib)
        self.filenames = test_lib[:,0]
        #self.labels = test_lib[:,1]
        self.transform = transform
        self.transform2 = transform2

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img = Image.open(path+"/test/"+format(self.filenames[idx])+'.png')  # PIL image
        #image= image.filter(ImageFilter.EDGE_ENHANCE)
        #image2 = image.filter(ImageFilter.FIND_EDGES)
        image = self.transform(img)
        image2 = self.transform2(img)
        #return image,self.filenames[idx]
        return image,image2,self.filenames[idx]

In [28]:
#trainA_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2),batch_size=128, shuffle=True,num_workers=2, pin_memory=True)
#trainC_dataloader = DataLoader(trainDataset(train_c, data_transforms,data_transforms2),batch_size=32, shuffle=True,num_workers=2, pin_memory=True)
#trainT_dataloader = DataLoader(trainDataset(train_t, data_transforms,data_transforms2),batch_size=32, shuffle=True,num_workers=2, pin_memory=True)

In [29]:
score_dataloader = DataLoaderX(trainDataset(datafilter[:1000], data_transforms,data_transforms2),
                               batch_size=80, 
                               shuffle=False,
                               num_workers=0, 
                               pin_memory=True)

In [30]:
test_dataloader = DataLoaderX(testDataset(test, data_transforms,data_transforms2),
                              batch_size=80,
                              shuffle=False,
                              num_workers=0, 
                              pin_memory=True)

In [31]:
# plt.figure(figsize=(10,10))
# plt.subplot(2,2,1)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][0]).convert('RGB'))
# plt.subplot(2,2,2)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[15][1]).convert('RGB'))
# plt.subplot(2,2,3)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][0]).convert('RGB'))
# plt.subplot(2,2,4)
# plt.imshow(transforms.ToPILImage()(trainA_dataloader.dataset[29][1]).convert('RGB'))


In [32]:
#(np.array(trainC_dataloader.dataset[4][0]).shape)

In [33]:
import pretrainedmodels

model = pretrainedmodels.__dict__["se_resnext101_32x4d"](num_classes=1000, pretrained='imagenet')
model.last_linear = nn.Linear(in_features=2048, out_features=lable_length, bias=True)

if torch.cuda.device_count() > 1:
    # device_ids has a default : all
    model = torch.nn.DataParallel(model, device_ids=[0,1,2,3]) 

model.to(device)

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [34]:
from torchsummary import summary
summary(model, (3, 220, 220))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 110, 110]           9,408
       BatchNorm2d-2         [-1, 64, 110, 110]             128
              ReLU-3         [-1, 64, 110, 110]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5          [-1, 128, 55, 55]           8,192
       BatchNorm2d-6          [-1, 128, 55, 55]             256
              ReLU-7          [-1, 128, 55, 55]               0
            Conv2d-8          [-1, 128, 55, 55]           4,608
       BatchNorm2d-9          [-1, 128, 55, 55]             256
             ReLU-10          [-1, 128, 55, 55]               0
           Conv2d-11          [-1, 256, 55, 55]          32,768
      BatchNorm2d-12          [-1, 256, 55, 55]             512
           Conv2d-13          [-1, 256, 55, 55]          16,384
      BatchNorm2d-14          [-1, 256,

In [35]:
# !ls ~/.torch/models  

In [36]:
train_loss_record = []
val_loss_record = []

In [37]:
def train(epoch,fnum,dloader):
    model.train()
    loss_avg = []
    for step, (x,y) in enumerate(dloader):
        data = Variable(x).cuda(async=True)   # batch x
        #data2 = Variable(x2).cuda()
        target = Variable(y).cuda(async=True)   # batch y
        #print(data.size())
        #print(data2.size())
        #print(target.size())
        #output = model(data,data2)
        #print(data.size())
        output = model(data)
        #print("out",output.size())
        #print("target",target.size())

        loss = loss_func(output, target.float())   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients
        if step==0:
            start = time.time()
            #break
            ti = 0
        elif step==100:
            ti = time.time()-start #total time = ti*(length/100)
            #print(ti)
            ti = ti*(len(dloader)/100)
        if step % 100 == 0:
            loss_avg.append(loss.data.item())
            
            second = ti*(((len(dloader)-step)/len(dloader)))#*(5-epoch)*(4-fnum)
            print('Train Fold:{}/4  Ep: {}/5 [{}/{} ({:.0f}%)]\t Loss: {:.6f}\t Remain : {} '.
                     format(fnum+1,
                            epoch+1, 
                            step * len(data), 
                            len(dloader.dataset),
                            100.*step/len(dloader), 
                            loss.data.item(),
                            datetime.timedelta(seconds = int(second))))
        #data.cpu()
        #data2.cpu()
        #target.cpu()
        torch.cuda.empty_cache()
    train_loss_record.append(np.array(loss_avg).sum()/len(loss_avg))
    print("Finish")

In [38]:
def val(dloader):
    model.eval()
    los = []
    for step, (x, y) in enumerate(dloader):
        data = Variable(x).cuda(async=True)
        #data2 = Variable(x2).cuda()
        target = Variable(y).cuda(async=True)
        
        #output = model(data,data2)
        with torch.no_grad():
            output = model(data)
        
        loss = loss_func(output, target.float())
        los.append(loss.item())
        
        
        if step %100 == 0:
            print('[{}/{} ({:.1f}%)]'.format(step * len(data), 
                                        len(dloader.dataset),
                                        100.*step/len(dloader)))
        #data.cpu()
        #data2.cpu()
        #arget.cpu()
        torch.cuda.empty_cache()
    los = np.array(los)
    avg_val_loss = los.sum()/len(los)
    val_loss_record.append(avg_val_loss)
    print("Avg val loss:{}".format(avg_val_loss))
    #return ans,out
    

In [39]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma
        
    def forward(self, input, target):
        # Inspired by the implementation of binary_cross_entropy_with_logits
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})".format(target.size(), input.size()))

        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + ((-max_val).exp() + (-input - max_val).exp()).log()

        # This formula gives us the log sigmoid of 1-p if y is 0 and of p if y is 1
        invprobs = F.logsigmoid(-input * (target * 2 - 1))
        loss = (invprobs * self.gamma).exp() * loss
        
        return loss.mean()

In [40]:
fold = KFold(n_splits = 4, random_state = 10)
for fold_num, (trn_idx, val_idx) in enumerate(fold.split(datafilter)):
    Ftrain_dataloader = datafilter[trn_idx, :]
    Fval_dataloader = datafilter[val_idx, :]
    #print(trn_idx,val_idx)

    val_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2, val_idx, aug),
                                batch_size=80, 
                                shuffle=False,
                                num_workers=0, 
                                pin_memory=True)
    train_dataloader = DataLoader(dataset_h5("all_data_109237v1.h5", data_transforms2, trn_idx, aug),
                                  batch_size=80, 
                                  shuffle=True,
                                  num_workers=0, 
                                  pin_memory=True)
    for epoch in range(5):
        ###########################################
        if epoch==0:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=0.00008,momentum=0.9, weight_decay=1e-4)
        #optimizer = torch.optim.Adam(model.parameters(), lr=0.00002/(2**epoch))
        #optimizer = torch.optim.SGD(net.parameters(), lr=args.lr, momentum=0.9, weight_decay=1e-4)
        loss_func = FocalLoss(0.4)
        #loss_func = torch.nn.MSELoss()
        ###########################################
        train(epoch,fold_num,train_dataloader) 
        val(val_dataloader)

Load data from .h5 file...
Take 0:00:02 seconds
Load data from .h5 file...
Take 0:00:06 seconds
Train Fold:1/4  Ep: 1/5 [0/900 (0%)]	 Loss: 0.525659	 Remain : 0:00:00 


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()
#!nvidia-smi

In [ ]:
fig = plt.figure()
plt.plot(train_loss_record)

In [ ]:
fig = plt.figure()
plt.plot(val_loss_record)

In [ ]:
torch.save(model, 'net_se_resnext101_t3.pkl')
#model = torch.load('net.pkl')